In [ ]:
model1 = r"alignment\models\seed42_v1.pkl"
model2 = r"alignment\models\seed43_v1.pkl"

dataset = "cifar10"
batch_size = 64
num_workers = 4
channel = 'AWGN'

In [3]:
%cd ..

d:\File\Repos\Deep-JSCC-PyTorch


In [4]:
import os
from torch.utils.data import Dataset
import torch
from PIL import Image
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.optim as optim
from dataset import Vanilla
from tqdm import tqdm
import torch.nn as nn
import pickle
from channel import Channel
from model import DeepJSCC, _Encoder, _Decoder

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

# load data
if dataset == 'cifar10':
    transform = transforms.Compose([transforms.ToTensor(), ])
    train_dataset = datasets.CIFAR10(root='../dataset/', train=True,
                                        download=True, transform=transform)

    train_loader = DataLoader(train_dataset, shuffle=True,
                                batch_size=batch_size, num_workers=num_workers)
    test_dataset = datasets.CIFAR10(root='../dataset/', train=False,
                                    download=True, transform=transform)
    test_loader = DataLoader(test_dataset, shuffle=True,
                                batch_size=batch_size, num_workers=num_workers)
elif dataset == 'imagenet':
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Resize((128, 128))])  # the size of paper is 128
    print("loading data of imagenet")
    train_dataset = datasets.ImageFolder(root='./dataset/ImageNet/train', transform=transform)

    train_loader = DataLoader(train_dataset, shuffle=True,
                                batch_size=batch_size, num_workers=num_workers)
    test_dataset = Vanilla(root='./dataset/ImageNet/val', transform=transform)
    test_loader = DataLoader(test_dataset, shuffle=True,
                                batch_size=batch_size, num_workers=num_workers)
else:
    raise Exception('Unknown dataset')

100%|██████████| 170M/170M [05:59<00:00, 474kB/s] 


In [6]:
model1_fp = r'alignment\models\seed42_v1.pkl'
model2_fp = r'alignment\models\seed43_v1.pkl'
saved = r'out\checkpoints\CIFAR10_8_7.0_0.17_AWGN_11h35m08s_on_Mar_27_2025\epoch_999.pkl'
snr = 7

In [7]:
def load_from_checkpoint(path, snr):
    state_dict = torch.load(path, map_location=torch.device('cpu'))
    from collections import OrderedDict
    new_state_dict = OrderedDict()

    for k, v in state_dict.items():
        name = k.replace('module.','') # remove `module.`
        new_state_dict[name] = v

    file_name = os.path.basename(os.path.dirname(saved))
    c = file_name.split('_')[1]
    c = int(c)
    model = DeepJSCC(c=c, channel_type=channel, snr=snr)

    model.load_state_dict(new_state_dict)
    model.change_channel(channel, snr)

    return model

model1 = load_from_checkpoint(model1_fp, snr).encoder
model2 = load_from_checkpoint(model2_fp, snr).encoder

In [ ]:
class ModelOutputDataset(Dataset):
    def __init__(self, dataloader, model1, model2, device='cpu'):
        self.outputs = []

        # Ensure models are in eval mode
        model1.eval()
        model2.eval()

        # Move models to the correct device
        model1.to(device)
        model2.to(device)

        with torch.no_grad():
            for inputs, _ in tqdm(dataloader, desc="Computing model outputs", total=50000):
                inputs = inputs.to(device)

                out1 = model1(inputs)
                out2 = model2(inputs)

                for o1, o2 in zip(out1, out2):
                    self.outputs.append((o1.cpu(), o2.cpu()))

    def __len__(self):
        return len(self.outputs)

    def __getitem__(self, idx):
        return self.outputs[idx]

In [9]:
data = ModelOutputDataset(train_loader, model1, model2, device)

Computing model outputs: 100%|██████████| 782/782 [00:28<00:00, 27.59it/s] 


In [16]:
# why this shape?

data.__getitem__(0)[0].shape

torch.Size([16, 8, 8])